### Dataset B: usar edad numérica escalada completa de la víctima y droppear la del llamante (re hacer notebook)

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import classification_report, f1_score
import gc


In [2]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

##### Datos a usar
Dataset de llamados completo, limpio con reducción de variables hecha a mano: variables agrupadas por temática, variables eliminadas por poco informativas, variables reducidas: hecho lugar, llamado provincia, llamante vinculo, victima nacionalidad, y agresor conocido/no conocido. 


##### Pipeline

1. cargar dataset
2. reemplazar NS/NC en convive por NA
3. hacer modificaciones para svm. 
4. separar NA de convive, serán el dataset "no visto"
5. separar features de target
6. separar train-test 
7. armar modelo
8. evaluar


##### Modificaiones para SVM

1. pasar datetime a timestamp y escalar
2. eliminar edad de quien llama, eliminar faltantes en edad de la víctima, y escalar la edad de la víctima
3. pasar edad a categórica 
4. Encodeo:
   1. edad, momento del día y estación del año con un encoder ordinal
   2. escalar las features encodeadas con ordinal
   3. variables de SI/NO encodear para 1 y 0
   4. el resto con one hot encoder
5. borrar las no usadas

In [3]:
# 1. cargar dataset

llamados= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v3.xlsx'), parse_dates=['llamado_fecha_hora'])


In [4]:
# 2 reemplazar todos los NS/NC en convive por na

llamados.loc[:, 'victima_convive_agresor'] = llamados['victima_convive_agresor'].replace({'NS/NC': pd.NA})



3. Modificaciones para SVM



In [5]:
scaler = MinMaxScaler()
encoder = OrdinalEncoder()

In [7]:
# 1. timestamp-fecha_hora encodeado y escalado



llamados['timestamp_encoded'] = encoder.fit_transform(llamados[['llamado_fecha_hora']])
llamados['timestamp_encoded_scaled'] = scaler.fit_transform(llamados[['timestamp_encoded']])
    
llamados.timestamp_encoded_scaled

# sobra llamado_fecha_hora y timestamp_encoded
    



0        0.000000
1        0.000060
2        0.000121
3        0.000181
4        0.000181
           ...   
19138    0.999758
19139    0.999940
19140    0.999819
19141    0.999879
19142    0.999879
Name: timestamp_encoded_scaled, Length: 19143, dtype: float64

In [8]:
# escalar edad, ver qué pasa con casos vacíos
llamados['victima_edad_escalada'] = scaler.fit_transform(llamados[['victima_edad']])

In [12]:
# 4. Variables categóricas que serán encodeadas con ordinal: momento día (ordinal), estación del año (ordinal).

llamados['momento_encoded'] = encoder.fit_transform(llamados[['momento_dia']])
llamados['estacion_encoded'] = encoder.fit_transform(llamados[['estacion_del_año']])



In [13]:

# 4.2 escalar las variables ordinales edad, momento del día y estación

llamados['momento_encoded_scaled'] = scaler.fit_transform(llamados[['momento_encoded']])
llamados['estacion_encoded_scaled'] = scaler.fit_transform(llamados[['estacion_encoded']])


#sobra momento_encoded y estacion_encoded
# momento_dia y estacion_del_año llamante_edad  llamado_fecha_hora y timestamp_encoded  


In [14]:
# droppear las columnas que no voy a usar ver qué pasó con víctima edad

llamados.drop([ 'momento_encoded', 'estacion_encoded', 'momento_dia',
                 'estacion_del_año', 'victima_edad', 'llamante_edad',  
                 'llamado_fecha_hora', 'timestamp_encoded'], axis=1, inplace=True) 

In [18]:
objetos = {}
for column in llamados.columns:
    if llamados[column].dtypes == object:
        objetos[column] = llamados[column].unique()

In [19]:
# 4.3 Variables de SI/NO encodear con dummy

si_no_columns = []
cat_mas_de_dos = []
for column in llamados.columns:
    if llamados[column].dtypes == object:
        if set(llamados[column].unique()).issubset({'SI', 'NO'}):
            si_no_columns.append(column)
        else:
            cat_mas_de_dos.append(column)

In [20]:
len(si_no_columns)

20

In [21]:
encodear_one_hot = ['llamante_genero',
 'caso_judicializado',
 'victima_genero',
 'victima_discapacidad',
 'agresor_conocido_no_conocido_red',
 'vinculo_llamante_red',
 'hecho_lugar_red',
 'llamado_provincia_red',
 'victima_nacionalidad_red']

In [22]:
# encodear si/no 

for col in si_no_columns:
    llamados[col] = llamados[col].map({'SI': 1, 'NO':0})

In [23]:
#encodear target
llamados['victima_convive_agresor'] = llamados['victima_convive_agresor'].map({'SI': 1, 'NO':0})

In [24]:
# encodear con one hot 

encoder = OneHotEncoder(sparse_output=True) # considerar drop first is sparse false

# Fit - transform 
encoded_sparse = encoder.fit_transform(llamados[encodear_one_hot])

# Convert the sparse matrix to a dense array
encoded_array = encoded_sparse.toarray()

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(encodear_one_hot))

# Drop the original columns that were one-hot encoded
llamados.drop(columns=encodear_one_hot, inplace=True)

# Concatenate the original DataFrame with the encoded DataFrame
llamados = pd.concat([llamados, encoded_df], axis=1)



In [25]:
# sacar todos los casos vacíos de convive y ponerlos en un nuevo dataset aparte, 
# será el test final a predecir con el mejor modelo


test_final = llamados[llamados['victima_convive_agresor'].isna()]


llamados = llamados.dropna(subset=['victima_convive_agresor'])


In [26]:
llamados = llamados.dropna(subset=['victima_edad_escalada'])

In [ ]:
gc.collect()

In [27]:
llamados.isnull().values.any()

False

In [28]:
# Separa features de target
X = llamados.drop(columns=['victima_convive_agresor'])
y = llamados['victima_convive_agresor']

# Split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Define GridSearchCV
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

# SVM init
svm = SVC()

# GridSearchCV 
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# mejor modelo
best_model = grid_search.best_estimator_

# predecir test set
y_pred = best_model.predict(X_test)

# Evaluar
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation f1: ", grid_search.best_score_)
print("Test set f1: ", f1_score(y_test, y_pred, average='weighted'))
print("Classification report:\n", classification_report(y_test, y_pred))


Best parameters found:  {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
Best cross-validation f1:  0.8693694473088158
Test set f1:  0.8684343625367262
Classification report:
               precision    recall  f1-score   support

         0.0       0.90      0.95      0.93      2334
         1.0       0.71      0.54      0.61       524

    accuracy                           0.88      2858
   macro avg       0.81      0.74      0.77      2858
weighted avg       0.87      0.88      0.87      2858



In [30]:
# mejor modelo al test ciego
test_final = test_final.dropna(subset=['victima_edad_escalada'])
X_na = test_final.drop(columns=['victima_convive_agresor'])

na_predictions = best_model.predict(X_na)

# df de predicciones
test_final['victima_convive_agresor_pred'] = na_predictions

print("\nPredictions for NA 'convive' values:")
print(test_final)


Predictions for NA 'convive' values:
       victima_a_resguardo  victima_convive_agresor  vs_tocamiento_sexual  \
10                       1                      NaN                     0   
12                       1                      NaN                     0   
13                       1                      NaN                     0   
14                       1                      NaN                     0   
16                       1                      NaN                     0   
...                    ...                      ...                   ...   
19097                    1                      NaN                     0   
19108                    1                      NaN                     0   
19113                    1                      NaN                     0   
19121                    1                      NaN                     0   
19129                    1                      NaN                     0   

       vs_intento_tocamiento  vs_groo

In [31]:
test_final.victima_convive_agresor_pred.value_counts()

victima_convive_agresor_pred
0.0    2670
1.0     307
Name: count, dtype: int64

In [32]:
test_final.shape

(2977, 65)

In [33]:
llamados.shape

(14286, 64)

In [34]:
llamados.victima_convive_agresor.value_counts()

victima_convive_agresor
0.0    11669
1.0     2617
Name: count, dtype: int64

In [35]:
print('Proporción de SI en dataset original', llamados.victima_convive_agresor.value_counts()[1]/llamados.shape[0], 'Proporción de SI predichos',test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0] )
print('Proporción de NO en dataset original', llamados.victima_convive_agresor.value_counts()[0]/llamados.shape[0], 'Proporción de NO predichos', test_final.victima_convive_agresor_pred.value_counts()[0]/test_final.shape[0])


Proporción de SI en dataset original 0.18318633627327455 Proporción de SI predichos 0.10312395028552233
Proporción de NO en dataset original 0.8168136637267255 Proporción de NO predichos 0.8968760497144777
